!mkdir cifar10
!curl -o cifar-10-python.tar.gz https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvzf cifar-10-python.tar.gz -C cifar10

### Сначала пытался сделать свою сетку, следуя советам в ноутбуке домашки. Точность доходила до 85-86 %. Поэтому пришлось мало-помалу красть сеточку из Интернета. 
# Итого 92.51% !
### Брал ее отсюда: https://github.com/Lasagne/Recipes/blob/master/papers/deep_residual_learning/Deep_Residual_Learning_CIFAR-10.py
### Используется Data Augmentation (добавляем к картинке рамку. В оригинальной статье ее заполняли 0, я решил сделать mode='median', показалось, что это логичнее), куча слоев и preprocessing (добавляем к датасету отраженные картинки). Дважды за время обучения уменьшаем learning rate в 10 раз. Дало нужный результат. (Пробовал уменьшать на других эпохах, получалось не очень)

In [28]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [29]:
import sys
import os
import time
import string
import random
import pickle

import numpy as np
import theano
import theano.tensor as T
import lasagne

sys.setrecursionlimit(10000)

def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = pickle.load(fo)
    fo.close()
    return dict

def load_data():
    xs = []
    ys = []
    for j in range(5):
      d = unpickle('cifar10/cifar-10-batches-py/data_batch_' + str(j+1))
      x = d['data']
      y = d['labels']
      xs.append(x)
      ys.append(y)

    d = unpickle('cifar10/cifar-10-batches-py/test_batch')
    xs.append(d['data'])
    ys.append(d['labels'])

    x = np.concatenate(xs) / np.float32(255)
    y = np.concatenate(ys)
    x = np.dstack((x[:, :1024], x[:, 1024:2048], x[:, 2048:]))
    x = x.reshape((x.shape[0], 32, 32, 3)).transpose(0,3,1,2)

    pixel_mean = np.mean(x[0:50000],axis=0)
    x -= pixel_mean

    X_train = x[0:50000,:,:,:]
    Y_train = y[0:50000]
    X_train_flip = X_train[:,:,:,::-1]
    Y_train_flip = Y_train
    X_train = np.concatenate((X_train,X_train_flip),axis=0)
    Y_train = np.concatenate((Y_train,Y_train_flip),axis=0)

    X_test = x[50000:,:,:,:]
    Y_test = y[50000:]

    return (
        lasagne.utils.floatX(X_train),
        Y_train.astype('int32'),
        lasagne.utils.floatX(X_test),
        Y_test.astype('int32'),
    )

In [30]:
X_train, y_train, X_test, y_test = load_data()

In [31]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *

input_X = T.tensor4("X")
target_y = T.vector("target Y integer",dtype='int32')

In [32]:
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import ElemwiseSumLayer
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import GlobalPoolLayer
from lasagne.layers import PadLayer
from lasagne.layers import ExpressionLayer
from lasagne.layers import NonlinearityLayer
from lasagne.nonlinearities import softmax, rectify
from lasagne.layers import batch_norm

def build_cnn(input_var=None, n=5):
    def residual_block(l, increase_dim=False, projection=False):
        input_num_filters = l.output_shape[1]
        if increase_dim:
            first_stride = (2,2)
            out_num_filters = input_num_filters * 2
        else:
            first_stride = (1,1)
            out_num_filters = input_num_filters

        stack_1 = batch_norm(ConvLayer(l, num_filters=out_num_filters, filter_size=(3,3), stride=first_stride, nonlinearity=rectify, pad='same', W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
        stack_2 = batch_norm(ConvLayer(stack_1, num_filters=out_num_filters, filter_size=(3,3), stride=(1,1), nonlinearity=None, pad='same', W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
        
        if increase_dim:
            if projection:
                projection = batch_norm(ConvLayer(l, num_filters=out_num_filters, filter_size=(1,1), stride=(2,2), nonlinearity=None, pad='same', b=None, flip_filters=False))
                block = NonlinearityLayer(ElemwiseSumLayer([stack_2, projection]),nonlinearity=rectify)
            else:
                identity = ExpressionLayer(
                    l, lambda X: X[:, :, ::2, ::2], lambda s: (s[0], s[1], s[2]//2, s[3]//2)
                )
                padding = PadLayer(identity, [out_num_filters//4,0,0], batch_ndim=1)
                block = NonlinearityLayer(ElemwiseSumLayer([stack_2, padding]),nonlinearity=rectify)
        else:
            block = NonlinearityLayer(ElemwiseSumLayer([stack_2, l]),nonlinearity=rectify)
        
        return block

    l_in = InputLayer(shape=(None, 3, 32, 32), input_var=input_var)

    l = batch_norm(ConvLayer(l_in, num_filters=16, filter_size=(3,3), stride=(1,1), nonlinearity=rectify, pad='same', W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
    
    for _ in range(n):
        l = residual_block(l)

    l = residual_block(l, increase_dim=True)
    for _ in range(1,n):
        l = residual_block(l)

    l = residual_block(l, increase_dim=True)
    for _ in range(1,n):
        l = residual_block(l)
    
    l = GlobalPoolLayer(l)

    network = DenseLayer(
            l, num_units=10,
            W=lasagne.init.HeNormal(),
            nonlinearity=softmax)

    return network

In [33]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False, augment=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        if augment:
            padded = np.pad(inputs[excerpt],((0,0),(0,0),(4,4),(4,4)),mode='median')
            random_cropped = np.zeros(inputs[excerpt].shape, dtype=np.float32)
            crops = np.random.random_integers(0,high=8,size=(batchsize,2))
            for r in range(batchsize):
                random_cropped[r,:,:,:] = padded[r,:,crops[r,0]:(crops[r,0]+32),crops[r,1]:(crops[r,1]+32)]
            inp_exc = random_cropped
        else:
            inp_exc = inputs[excerpt]

        yield inp_exc, targets[excerpt]

In [34]:
net = build_cnn(input_X)

In [35]:
y_predicted = lasagne.layers.get_output(net)
all_weights = lasagne.layers.get_all_params(net, trainable=True)
all_layers = lasagne.layers.get_all_layers(net)
print(all_weights)

[W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, b]


In [36]:
reg_lambda = 0.0001
current_learning_rate = 0.1
loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean()
loss += reg_lambda * lasagne.regularization.regularize_layer_params(all_layers, lasagne.regularization.l2)
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, target_y).mean()

In [37]:
grad_update = lasagne.updates.momentum(loss, all_weights, learning_rate=current_learning_rate, momentum=0.9)

In [38]:
train_fun = theano.function([input_X, target_y], [loss, accuracy], updates=grad_update, allow_input_downcast=True)

In [ ]:
accuracy_fun = theano.function([input_X, target_y], accuracy, allow_input_downcast=True)

In [ ]:
import time

epohs_number = 82
batch_size = 128
batch_counter = 0

for epoch in range(epohs_number):
    batch_counter += 1
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size, shuffle=True, augment=True):
        inputs, targets = batch
        train_err_batch, train_acc_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1
    
    loss_history.append(train_err / train_batches)

    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    print("Epoch {} of {} took {:.3f}s".format(batch_counter, epohs_number, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))

    if (epoch + 1) == 41 or (epoch + 1) == 61:
        current_learning_rate = current_learning_rate * 0.1
        print("New LR: {}".format(current_learning_rate))
        grad_update = lasagne.updates.momentum(
            loss, all_weights, learning_rate=current_learning_rate, momentum=0.9
        )
        train_fun = theano.function(
            [input_X, target_y], [loss, accuracy], updates=grad_update, allow_input_downcast=True
        )
        accuracy_fun = theano.function([input_X, target_y], accuracy, allow_input_downcast=True)

/home/ec2-user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: DeprecationWarning: This function is deprecated. Please call randint(0, 8 + 1) instead


Epoch 1 of 82 took 212.418s
  training loss (in-iteration):		1.850246
  train accuracy:		44.16 %
  validation accuracy:		61.19 %
Epoch 2 of 82 took 212.414s
  training loss (in-iteration):		1.188111
  train accuracy:		67.98 %
  validation accuracy:		75.73 %
Epoch 3 of 82 took 212.435s
  training loss (in-iteration):		0.932953
  train accuracy:		76.65 %
  validation accuracy:		79.28 %
Epoch 4 of 82 took 212.415s
  training loss (in-iteration):		0.814125
  train accuracy:		80.42 %
  validation accuracy:		80.40 %
Epoch 5 of 82 took 212.447s
  training loss (in-iteration):		0.747206
  train accuracy:		82.69 %
  validation accuracy:		81.71 %
Epoch 6 of 82 took 212.430s
  training loss (in-iteration):		0.710085
  train accuracy:		83.94 %
  validation accuracy:		83.33 %
Epoch 7 of 82 took 212.457s
  training loss (in-iteration):		0.677047
  train accuracy:		85.06 %
  validation accuracy:		83.91 %
Epoch 8 of 82 took 212.402s
  training loss (in-iteration):		0.660051
  train accuracy:		85.65 %


In [42]:
print(all_weights)

[W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, beta, gamma, W, b]


In [ ]:
plt.plot([i for i in range(len(loss_history))], loss_history)

In [43]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 92.5:
    print("Achievement unlocked: колдун 80 уровня")
else:
    print("Нужно больше магии!")

Final results:
  test accuracy:		92.51 %
Achievement unlocked: колдун 80 уровня
